In [5]:
from huggingface_hub import notebook_login
notebook_login()

In [7]:
from huggingface_hub import HfApi

api = HfApi()
user = api.whoami()
models = api.list_models()

### Searching for models

In [8]:
models = api.list_models(
    task = "text-to-image",
    author = "CompVis",
    tags="diffusers:StableDiffusionPipeline",
    sort="downloads",

)
top_models = list(models)[0]
print(top_models)

C:\Users\USER\AppData\Roaming\Python\Python313\site-packages\huggingface_hub\utils\_deprecation.py:100: FutureWarning: Deprecated argument(s) used in 'list_models': task, tags. Will not be supported from version '1.0'.

Use `filter` instead.
  warnings.warn(message, FutureWarning)


ModelInfo(id='CompVis/stable-diffusion-v1-4', author=None, sha=None, created_at=datetime.datetime(2022, 8, 20, 13, 26, 13, tzinfo=datetime.timezone.utc), last_modified=None, private=False, disabled=None, downloads=523899, downloads_all_time=None, gated=None, gguf=None, inference=None, inference_provider_mapping=None, likes=6968, library_name='diffusers', tags=['diffusers', 'safetensors', 'stable-diffusion', 'stable-diffusion-diffusers', 'text-to-image', 'arxiv:2207.12598', 'arxiv:2112.10752', 'arxiv:2103.00020', 'arxiv:2205.11487', 'arxiv:1910.09700', 'license:creativeml-openrail-m', 'endpoints_compatible', 'diffusers:StableDiffusionPipeline', 'region:us'], pipeline_tag='text-to-image', mask_token=None, card_data=None, widget_data=None, model_index=None, config=None, transformers_info=None, trending_score=None, siblings=None, spaces=None, safetensors=None, security_repo_status=None, xet_enabled=None)


### Using Models from the API

In [9]:
top_models_id = top_models.id
print(top_models_id)

CompVis/stable-diffusion-v1-4


In [10]:
# Find the list of models for a task
models = api.list_models(task="text-to-image")
print(f"Task: text-to-image, Models: {len(list(models))}")

C:\Users\USER\AppData\Roaming\Python\Python313\site-packages\huggingface_hub\utils\_deprecation.py:100: FutureWarning: Deprecated argument(s) used in 'list_models': task. Will not be supported from version '1.0'.

Use `filter` instead.
  warnings.warn(message, FutureWarning)


Task: text-to-image, Models: 82442


In [11]:
models = api.list_models(
    # Filter for text-to-image tasks
    task="text-to-image",
    author="CompVis",
    # Filter for models that can be loaded by the diffusers library
    tags="diffusers:StableDiffusionPipeline",
    # Sort according to the most popular
    sort="likes"
)

models = list(models)

# Load the most popular model from models
pipe = StableDiffusionPipeline.from_pretrained(models[0].id)

NameError: name 'StableDiffusionPipeline' is not defined

### Preprocessing different modalities

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

text = "Do you need more "

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

C:\Users\USER\AppData\Roaming\Python\Python313\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\USER\.cache\huggingface\hub\models--distilbert--distilbert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [16]:
text = "Do you need more information about Hugging Face ɡo͝od môrniNG ?"

print(tokenizer.backend_tokenizer.normalizer.normalize_str(text))

do you need more information about hugging face ɡood morning ?


In [17]:
tokenizer(text, return_tensors="pt", padding=True)

{'input_ids': tensor([[  101,  2079,  2017,  2342,  2062,  2592,  2055, 17662,  2227,  1116,
         17139,  2851,  1029,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

#### Preprocessing images
 - Normalization: pixel intensity updates
 - Resize: Match input layer of model
 - General rule => use preprocessing of original model

  - Multimodal tasks require consistent preprocessing

In [18]:
from transformers import BlipProcessor, BlipForConditionalGeneration
checkpoint = "Salesforce/blip-image-captioning-base"
model = BlipForConditionalGeneration.from_pretrained(checkpoint)
processor = BlipProcessor.from_pretrained(checkpoint)

config.json: 0.00B [00:00, ?B/s]

C:\Users\USER\AppData\Roaming\Python\Python313\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\USER\.cache\huggingface\hub\models--Salesforce--blip-image-captioning-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling 

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Encode image -> transform to text encoding -> decode text

In [19]:
image = load_dataset("nlphuji/flick30k")['test'][11]['image']
inputs = processor(image, return_tensors="pt")
output = model.generate(**inputs)
print(processor.decode(output[0]))


NameError: name 'load_dataset' is not defined

In [21]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")
text = "Do you need more information about Hugging Face ɡo͝od môrniNG ?"

print(tokenizer.backend_tokenizer.normalizer.normalize_str(text))

do you need more information about hugging face ɡood morning ?


In [22]:
tokenizer(text, return_tensors="pt", padding=True)

{'input_ids': tensor([[  101,  2079,  2017,  2342,  2062,  2592,  2055, 17662,  2227,  1116,
         17139,  2851,  1029,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

#### Preprocessing images

Encode images -> transform to text encoding -> decode text

In [23]:
from transformers import BlipProcessor, BlipForConditionalGeneration
checkpoint = "Salesforce/blip-image-captioning-base"
model = BlipForConditionalGeneration.from_pretrained(checkpoint)
processor = BlipProcessor.from_pretrained(checkpoint)

In [26]:
from datasets import load_dataset, Audio

In [28]:
image = load_dataset("nlphuji/flickr30k")['test'][11]['image']
inputs = processor(image, return_tensors="pt")
output = model.generate(**inputs)
print(processor.decode(output[0]))

README.md:   0%|          | 0.00/641 [00:00<?, ?B/s]

C:\Users\USER\AppData\Roaming\Python\Python313\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\USER\.cache\huggingface\hub\datasets--nlphuji--flickr30k. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


flickr30k.py: 0.00B [00:00, ?B/s]

RuntimeError: Dataset scripts are no longer supported, but found flickr30k.py

#### Preprocessing Audio
- Audio preprocessing:
    - Sequential array -> filter/padding
    - Sampling rate -> resampling

Feature extraction as model input (spectrogram)

In [29]:
from datasets import load_dataset, Audio
dataset = load_dataset("CSTR-Edinburgh/vctk")['train']
dataset = dataset.cast_column("audio", Audio(sampling_rate=16_000))
print(dataset[0]['audio'])

README.md: 0.00B [00:00, ?B/s]

C:\Users\USER\AppData\Roaming\Python\Python313\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\USER\.cache\huggingface\hub\datasets--CSTR-Edinburgh--vctk. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vctk.py: 0.00B [00:00, ?B/s]

RuntimeError: Dataset scripts are no longer supported, but found vctk.py

In [30]:
from transformers import AutoProcessor
precessor = AutoProcessor.from_pretrained("openai/whisper-small")
audio_pp = processor(dataset[0]['audio']['array'], sampling_rate=16_000, return_tensors='pt')

preprocessor_config.json: 0.00B [00:00, ?B/s]

C:\Users\USER\AppData\Roaming\Python\Python313\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\USER\.cache\huggingface\hub\models--openai--whisper-small. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10

tokenizer_config.json: 0.00B [00:00, ?B/s]

'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 7e7fd342-337c-4cac-b936-8a85d5737d83)')' thrown while requesting GET https://huggingface.co/api/resolve-cache/models/openai/whisper-small/973afd24965f72e36ca33b3055d56a652f456b4d/vocab.json
Retrying in 1s [Retry 1/5].


vocab.json: 0.00B [00:00, ?B/s]

...

merges.txt: 0.00B [00:00, ?B/s]

normalizer.json: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

NameError: name 'dataset' is not defined

In [31]:
# Load the first caption from the image at index 5
text = dataset[5]["caption"][0]
print(text)

NameError: name 'dataset' is not defined

In [32]:
# Load the first caption from the image at index 5
text = dataset[5]["caption"][0]
print(text)

# Load the tokenizer of the pretrained model
tokenizer = AutoTokenizer.from_pretrained('distilbert/distilbert-base-uncased')
# Perform full preprocessing of the text
encoded_input = tokenizer(text, return_tensors='pt')
print(encoded_input)

NameError: name 'dataset' is not defined

In [33]:
# Load the image from index 5 of the dataset
image = dataset[5]["image"]

# Load the image processor of the pretrained model
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")

# Preprocess the image
inputs = processor(images=image, return_tensors="pt")

# Generate a caption using the model
output = model.generate(**inputs)
print(f'Generated caption: {processor.decode(output[0])}')
print(f'Original caption: {dataset[5]["caption"][0]}')

NameError: name 'dataset' is not defined

In [34]:
# Resample the audio to a frequency of 16,000 Hz
dataset = dataset.cast_column("audio", Audio(sampling_rate=16_000))

# Load the audio processor
processor = AutoProcessor.from_pretrained('openai/whisper-small')

# Preprocess the audio data of the 0th dataset element
audio_pp = processor(dataset[0]["audio"]["array"], sampling_rate=16_000, padding=True, return_tensors="pt")
make_spectrogram(audio_pp["input_features"][0])

NameError: name 'dataset' is not defined